FFT windowing
---


This notebook visualizes the FFT windows with different windowing functions. To start working with this notebook, you need to download the [IEEE dataset](https://ieee-dataport.org/open-access/experimental-database-detecting-and-diagnosing-rotor-broken-bar-three-phase-induction), unpack it and place in the folder named *IEEE* above the current folder, containing the notebook. Also, you should have the libraries from *requirements.txt* installed in the virtual environment, as described in the README file.

The plots are made with interactive module to enable the inspection of the window cycle number, motor state and load on the current signal frequency spectrum.

#### Libraries import

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import h5py
import scipy.fft
from ipywidgets import interact
import ipywidgets as widgets

#### Data import

In [ ]:
# array of data[class[0BRB-4BRB]][load[0.125-1]]
# data[0][0] - 0BRB, 12.5% load
# data[3][2] - 3BRB, 37.5% load etc.
exp_data=[[j for j in range(8)] for i in range(5)]
load_levels=['torque05','torque10','torque15','torque20','torque25','torque30','torque35','torque40']
states=['rs','r1b','r2b','r3b','r4b']
i=0
F_sampling=50000 #Hz, current value sampling
for state in states:
    j=0
    for level in load_levels:
        mat=h5py.File('../IEEE/struct_'+state+'_R1.mat','r')
        dataset_refname=mat[state][level]['Ia'][0]
        data=mat[dataset_refname[0]]
        data=np.ravel(data)
        data=data[120000:] # discards the transient part of the signals
        d={'Current, A':data}
        exp_data[i][j]=pd.DataFrame(data=d,index=np.ravel([k/F_sampling for k in range (len(data))]))
        exp_data[i][j].index.name='Time, s'
        j+=1
    mat.close()
    i+=1

In [ ]:
print(f'Shape of the data: {np.shape(exp_data)}')

In [ ]:
exp_data[0][0].head() #array of data[class[0BRB-3BRB]][load[0-1]]

#### Current signal

In [ ]:
style = {'description_width': 'initial'}
@interact(brb=widgets.IntSlider(description='BRB',style=style,min=0,max=4,step=1,value=0),
          load=widgets.FloatSlider(description='Load level, %',style=style,min=12.5,max=100,step=12.5,value=12.5))
def current_signal(brb,load):
    load=int(load/12.5-1)
    plt.rcParams['figure.dpi']=300
    fig, ax=plt.subplots(figsize=(16,7))
    plt.plot(exp_data[brb][load]['Current, A'][:0.06],linewidth=1)
    ax.set_xlabel('Time, s', fontsize=18)
    ax.set_ylabel('Current, A', fontsize=18)

### Apply windowing

#### Rectangular

In [ ]:
style = {'description_width': 'initial'}
@interact(brb=widgets.IntSlider(description='BRB',style=style,min=0,max=4,step=1,value=0),
          load=widgets.FloatSlider(description='Load level, %',style=style,min=12.5,max=100,step=12.5,value=12.5),
          window_length=widgets.IntSlider(description='Window length',style=style,min=1,max=200,step=1,value=40))
def fft_hamming(brb,load,window_length):
    #signal info
    F_sampling=50000 #Hz
    resolution=1/F_sampling
    F_grid=60 #Hz, grid frequency
    load=int(load/12.5-1)
    data=exp_data[brb][load][:window_length/F_grid].copy()
    N=np.shape(data)[0]
    ywf_rect=scipy.fft.rfft(np.ravel(data['Current, A']))
    yfreq=scipy.fft.fftfreq(N,d=resolution)[:N//2]
    fig,ax=plt.subplots(figsize=(16,7))
    plt.semilogy(yfreq[1:N//2],2/N*np.abs(ywf_rect[1:N//2]));
    ax.set_xlim([0,120])
    ax.set_xlabel('Frequency, Hz',fontsize=18)
    ax.set_ylabel('Amplitude, dB',fontsize=18)
    ax.set_title('Rectangular window',fontsize=20)

#### Blackman

In [ ]:
style = {'description_width': 'initial'}
@interact(brb=widgets.IntSlider(description='BRB',style=style,min=0,max=4,step=1,value=0),
          load=widgets.FloatSlider(description='Load level, %',style=style,min=12.5,max=100,step=12.5,value=12.5),
          window_length=widgets.IntSlider(description='Window length',style=style,min=1,max=200,step=1,value=40))
def fft_blackman(brb,load,window_length):
    #signal info
    plt.rcParams['figure.dpi']=300
    F_sampling=50000 #Hz
    resolution=1/F_sampling
    F_s=60 #supply frequency
    load=int(load/12.5-1)
    data=exp_data[brb][load][:window_length/F_s].copy()
    window=scipy.signal.windows.blackman(len(data))
    #signal length
    N=np.shape(data)[0]
    ywf=scipy.fft.fft(np.ravel(data['Current, A'])*window)
    yfreq=scipy.fft.fftfreq(N,d=resolution)[:N//2]
    fig,ax=plt.subplots(figsize=(16,7));
    plt.semilogy(yfreq[1:N//2],2/N*np.abs(ywf[1:N//2]));
    ax.set_xlim([0,120])
    ax.set_xlabel('Frequency, Hz', fontsize=18)
    ax.set_ylabel('Amplitude, dB', fontsize=18)
    ax.set_title('Blackman window',fontsize=20)

#### Hanning

In [ ]:
style = {'description_width': 'initial'}
@interact(brb=widgets.IntSlider(description='BRB',style=style,min=0,max=4,step=1,value=0),
          load=widgets.FloatSlider(description='Load level, %',style=style,min=12.5,max=100,step=12.5,value=12.5),
          window_length=widgets.IntSlider(description='Window length',style=style,min=1,max=200,step=1,value=40))
def fft_hanning(brb,load,window_length):
    #signal info
    F_sampling=50000 #Hz
    resolution=1/F_sampling
    F_s=60 #grid frequency
    load=int(load/12.5-1)
    data=exp_data[brb][load][:window_length/F_s].copy()
    N=np.shape(data)[0]
    window_hanning=np.hanning(len(data['Current, A']))
    ywf_hanning=scipy.fft.rfft(np.ravel(data['Current, A'])*window_hanning)
    yfreq=scipy.fft.fftfreq(N,d=resolution)[:N//2]
    fig,ax=plt.subplots(figsize=(16,7))
    plt.semilogy(yfreq[1:N//2],2/N*np.abs(ywf_hanning[1:N//2]))
    ax.set_xlim([0,120])
    ax.set_xlabel('Frequency, Hz',fontsize=18)
    ax.set_ylabel('Amplitude, dB',fontsize=18)
    ax.set_title(f'Hanning window',fontsize=20)

#### Hamming

In [ ]:
style = {'description_width': 'initial'}
@interact(brb=widgets.IntSlider(description='BRB',style=style,min=0,max=4,step=1,value=0),
          load=widgets.FloatSlider(description='Load level, %',style=style,min=12.5,max=100,step=12.5,value=12.5),
          window_length=widgets.IntSlider(description='Window length',style=style,min=1,max=200,step=1,value=40))
def fft_hamming(brb,load,window_length):
    #signal info
    F_sampling=50000 #Hz
    resolution=1/F_sampling
    F_s=60 #Hz, grid frequency
    load=int(load/12.5-1)
    data=exp_data[brb][load][:window_length/F_s].copy()
    N=np.shape(data)[0]
    window_hamming=np.hamming(len(data['Current, A']))
    ywf_hamming=scipy.fft.rfft(np.ravel(data['Current, A'])*window_hamming)
    yfreq=scipy.fft.fftfreq(N,d=resolution)[:N//2]
    fig,ax=plt.subplots(figsize=(16,7))
    plt.semilogy(yfreq[1:N//2],2/N*np.abs(ywf_hamming[1:N//2]))
    ax.set_xlim([0,120])
    ax.set_xlabel('Frequency, Hz',fontsize=18)
    ax.set_ylabel('Amplitude, dB',fontsize=18)
    ax.set_title('Hamming window',fontsize=20)

#### Flattop

In [ ]:
style = {'description_width': 'initial'}
@interact(brb=widgets.IntSlider(description='BRB',style=style,min=0,max=4,step=1,value=0),
          load=widgets.FloatSlider(description='Load level, %',style=style,min=12.5,max=100,step=12.5,value=12.5),
          window_length=widgets.IntSlider(description='Window length',style=style,min=1,max=200,step=1,value=40))
def fft_hamming(brb,load,window_length):
    #signal info
    F_sampling=50000 #Hz
    resolution=1/F_sampling
    F_s=60 #Hz, grid frequency
    load=int(load/12.5-1)
    data=exp_data[brb][load][:window_length/F_s].copy()
    N=np.shape(data)[0]
    window_flattop=scipy.signal.windows.flattop(len(data['Current, A']))
    ywf_flattop=scipy.fft.rfft(np.ravel(data['Current, A'])*window_flattop)
    yfreq=scipy.fft.fftfreq(N,d=resolution)[:N//2]
    fig,ax=plt.subplots(figsize=(16,7))
    plt.semilogy(yfreq[1:N//2],2/N*np.abs(ywf_flattop[1:N//2]))
    ax.set_xlim([0,120])
    ax.set_xlabel('Frequency, Hz',fontsize=18)
    ax.set_ylabel('Amplitude, dB',fontsize=18)
    ax.set_title('Flattop window',fontsize=20)